In [1]:
import json
import pandas as pd

In [4]:
frames_review = []
for chunk in pd.read_json('../data/YelpDatasetKaggle/yelp_academic_dataset_review.json', lines=True, chunksize=20000):
    frames_review.append(chunk)
review=pd.concat(frames_review)

In [5]:
frames = []
for chunk in pd.read_json('../data/YelpDatasetKaggle/yelp_academic_dataset_user.json', lines=True, chunksize = 10000):
    frames.append(chunk)
user = pd.concat(frames)

In [6]:
frames_business = []
for chunk in pd.read_json('../data/YelpDatasetKaggle/yelp_academic_dataset_business.json', lines=True, chunksize = 10000):
    frames_business.append(chunk)
business = pd.concat(frames_business)

In [7]:
import re
business['restaurant']=business['categories'].str.contains('Restaurants',flags=re.IGNORECASE)

In [8]:
business_restaurant=business[business['restaurant']==True]

In [9]:
business_restaurant.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,restaurant
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo...","{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",True
20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,0,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...","Food, Restaurants, Grocery, Middle Eastern",None,True
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,1,"{'RestaurantsTableService': 'False', 'Restaura...","Restaurants, Cheesesteaks, Poutineries","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",True
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,1,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Japanese, Fast Food, Food Court, Restaurants",None,True
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,1,"{'Ambience': '{'touristy': False, 'hipster': F...","Persian/Iranian, Turkish, Middle Eastern, Rest...","{'Tuesday': '12:0-21:0', 'Wednesday': '12:0-21...",True


In [10]:
business_restaurant.reset_index(drop=True).head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,restaurant
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Ethnic Food, Food Trucks, Specialty Food, Impo...","{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",True
1,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,0,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...","Food, Restaurants, Grocery, Middle Eastern",None,True
2,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,1,"{'RestaurantsTableService': 'False', 'Restaura...","Restaurants, Cheesesteaks, Poutineries","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",True
3,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,1,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Japanese, Fast Food, Food Court, Restaurants",None,True
4,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,1,"{'Ambience': '{'touristy': False, 'hipster': F...","Persian/Iranian, Turkish, Middle Eastern, Rest...","{'Tuesday': '12:0-21:0', 'Wednesday': '12:0-21...",True


In [11]:
business_restaurant.shape

(63944, 15)

In [13]:
business_restaurant.to_pickle('../data/all-restaurants/restaurants.pickle')

In [14]:
review_restaurant=review.loc[review['business_id'].isin(business_restaurant['business_id'].unique())]

In [15]:
review_restaurant.reset_index(drop=True).head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
1,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
2,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57
3,L2O_INwlrRuoX05KSjc4eg,5vD2kmE25YBrbayKhykNxQ,nlxHRv1zXGT0c0K51q3jDg,5,2,0,0,This is definitely my favorite fast food sub s...,2013-05-07 07:25:25
4,ZayJ1zWyWgY9S_TRLT_y9Q,aq_ZxGHiri48TUXJlpRkCQ,Pthe4qk5xh4n-ef-9bvMSg,5,1,0,0,"Really good place with simple decor, amazing f...",2015-11-05 23:11:05


In [21]:
review_restaurant.to_pickle('../data/all-restaurants/restaurant_review.pickle')

In [22]:
review_restaurant.shape

(1503844, 9)

In [23]:
user_restaurant=user.loc[user['user_id'].isin(review_restaurant['user_id'].unique())]

In [24]:
user_restaurant.to_pickle('../data/all-restaurants/restaurant_users.pickle')

In [25]:
user_in_vegas.shape

(536321, 22)

In [ ]:
categories_df = pd.read_csv('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\categories.csv')
cuisine = categories_df['cuisine'].tolist()
food = categories_df['food'].tolist()
rest_type = categories_df['rest_type'].tolist()
diet = categories_df['diet'].tolist()

In [ ]:
writer = open('../data/restaurants/kgnew.txt', 'w', encoding='utf-8')
for index, row in rest_df.iterrows():
    for c in row['categories'].split(", "):
        if c != 'Restaurants':
            if c in cuisine:
                writer.write('%s\t%s\t%s\n' % (row['business_id'], 'cuisine', c))
            elif c in food:
                writer.write('%s\t%s\t%s\n' % (row['business_id'], 'food', c))
            elif c in rest_type:
                writer.write('%s\t%s\t%s\n' % (row['business_id'], 'rest_type', c))
            elif c in diet:
                writer.write('%s\t%s\t%s\n' % (row['business_id'], 'diet', c))
            else:
                writer.write('%s\t%s\t%s\n' % (row['business_id'], 'business_type', c))
    writer.write('%s\t%s\t%s\n' % (row['business_id'], 'stars', row['stars']))
    writer.write('%s\t%s\t%s\n' % (row['business_id'], 'city', row['city']))
    writer.write('%s\t%s\t%s\n' % (row['business_id'], 'state', row['state']))
    writer.write('%s\t%s\t%s\n' % (row['business_id'], 'review_count', row['review_count']))
writer.close()